In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from resnet import ResNet18, ResNet34
%matplotlib inline

In [3]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

MAX_TRAINING = 500
MAX_VALIDATION = 50
MAX_TESTING = 50
correct_class = {1 : 'airplane', 2 : 'automobile', 3 : 'bird', 4 : 'cat', 5 : \
                 'deer', 6 : 'dog', 7 : 'frog', 8 : 'horse', 9 : 'ship', 10 : 'truck'}
architecture = 'ResNet18'
label, keywords = 'milestone-experiments', 'benchmark_testing'

#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()

x_train_max, y_train_max = get_random_data(x_train, y_train, 0, MAX_TRAINING*3, MAX_TRAINING)
x_val_max, y_val_max = get_random_data(x_train, y_train, MAX_TRAINING*3+1, x_train.shape[0], MAX_VALIDATION)
x_test_max, y_test_max = get_random_data(x_test, y_test, 0, x_test.shape[0], MAX_TESTING)

# Keeping the data between 0 and 1 
x_train, x_test = x_train_max / 255.0, x_test_max / 255.0
x_val, y_val = x_val_max/255.0, y_val_max/255.0
y_train = to_categorical(y_train_max, num_classes=10)
y_test = to_categorical(y_test_max, num_classes=10)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=500, Y=500
Number of Validation Samples: X=50, Y=50
Number of Test Samples: X=50, Y=50


In [7]:
img_rows, img_cols, img_channels = x_train[0].shape
nb_classes = y_test[0].shape[0]

model = ResNet18((img_rows, img_cols, img_channels), nb_classes, 'tanh') 
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy", "mae", "mse"])
model.fit(x_train, y_train, epochs=1)
score = model.evaluate(x_test, y_test)
print (score)

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Train on 500 samples
50/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
img_rows, img_cols, img_channels = x_train[0].shape
nb_classes = y_test[0].shape[0]

model = ResNet18((img_rows, img_cols, img_channels), nb_classes, 'relu') 
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy", "mae", "mse"])
model.fit(x_train, y_train, epochs=5, batch_size=250)
score = model.evaluate(x_test, y_test, batch_size=50)
print (score)

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Train on 500 samples
Epoch 1/5
500/500 [==============================] - 116s 232ms/sample - loss: 8.9411 - accuracy: 0.1280 - mae: 0.4515 - mse: 0.5589
Epoch 2/5
500/500 [==============================] - 4s 8ms/sample - loss: 6.4081 - accuracy: 0.3120 - mae: 0.6244 - mse: 1.1075
Epoch 3/5
500/500 [==============================] - 3s 7ms/sample - loss: 5.3666 - accuracy: 0.3720 - mae: 0.7594 - mse: 1.5602
Epoch 4/5
500/500 [==============================] - 21s 43ms/sample - loss: 4.7967 - accuracy: 0.4080 - mae: 0.8624 - mse: 2.0132
Epoch 5/5
500/500 [==============================] - 79s 157ms/sample - loss: 4.5827 - accuracy: 0.4360 - mae: 0.9531 - mse: 2.5133
